# Huggingface Ecosystem Overview

Huggingface is a company that has revolutionized the field of Natural Language Processing (NLP) by providing an open-source library and tools that facilitate the use of state-of-the-art models for various NLP tasks.

### Key Components of the Huggingface Ecosystem:

1. **Transformers**: A library that provides APIs and tools to easily download and fine-tune state-of-the-art pre-trained models.
2. **Datasets**: A library to access and process large datasets used for NLP and other machine learning tasks.
3. **PEFT (Parameter-Efficient Fine-Tuning)**: A library for efficient model fine-tuning using parameter-efficient techniques.
4. **Accelerate**: A library to accelerate PyTorch and TensorFlow models' training and deployment across multiple devices (GPUs).
5. **Huggingface Hub**: A central repository for pre-trained models, datasets, and metrics, allowing seamless sharing and collaboration.

Let's explore each component in detail with hands-on examples.

## Transformers Library

The `transformers` library by Huggingface is a powerful toolkit that provides state-of-the-art pre-trained models and easy-to-use APIs for NLP tasks such as text classification, named entity recognition, translation, text generation, and more.

### Key Features:
- Provides thousands of pre-trained models.
- Supports multiple frameworks: PyTorch, TensorFlow, and JAX.
- Easy integration with the Huggingface Hub.

### Example: Loading a Pre-trained Model and Running Inference
Let's load a pre-trained BERT model and use it for a simple text classification task.
For that we use `pipeline()`.
`pipeline()` is a very convenient way to use a pretrained model for inference. You can use the `pipeline()` out-of-the-box for many tasks across different modalities, as can be seen [here](https://huggingface.co/docs/transformers/quicktour).

In [1]:
from transformers import pipeline

# Load a pre-trained sentiment-analysis pipeline
classifier = pipeline("sentiment-analysis")

2024-09-25 10:41:34.040792: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-25 10:41:34.040882: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-25 10:41:34.041972: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-25 10:41:34.047420: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
No model was supplied, defaulted to

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [2]:
# Test the pipeline with some example text
result = classifier("Huggingface is an amazing platform for NLP research!")
print(result)

[{'label': 'POSITIVE', 'score': 0.9998051524162292}]


Feel free to try out different sentences, also negative ones.
Should you have more than one input, pass them as a list. You will then get a list of dictionaries as output.

In [3]:
result = classifier(["EuorCC courses are not bad at all. There is lots to gain from them.",
                    "I think Large Language Models are overrated."])
print(result)

[{'label': 'POSITIVE', 'score': 0.9970724582672119}, {'label': 'NEGATIVE', 'score': 0.99234539270401}]


You can see, that the model not anly predicts the sentiment, but also outputs a score, which is a probability that indicates the model's confidence in its prediction.

## Datasets Library

The `datasets` library provides a lightweight library for easily downloading and using datasets in NLP and other ML domains. It is optimized for both in-memory and out-of-memory (on-disk) use, making it suitable for handling very large datasets.

### Key Features:
- Access to thousands of datasets in various domains.
- Built-in data processing tools such as caching, shuffling, and batching.
- Easy integration with the `transformers` library for model training.

### Example: Loading and Exploring a Dataset
Let's load a sample dataset and explore its content.
The **IMDB** dataset is a popular benchmark dataset used for sentiment analysis tasks in natural language processing. It consists of movie reviews from the Internet Movie Database (IMDB) and is specifically designed for binary sentiment classification: determining whether a given movie review expresses a positive or negative sentiment.


In [4]:
from datasets import load_dataset

# Load the IMDB dataset
dataset = load_dataset("imdb")

In [5]:
# Display the 10th example in the training set
print(dataset['train'][9])

{'text': "This is said to be a personal film for Peter Bogdonavitch. He based it on his life but changed things around to fit the characters, who are detectives. These detectives date beautiful models and have no problem getting them. Sounds more like a millionaire playboy filmmaker than a detective, doesn't it? This entire movie was written by Peter, and it shows how out of touch with real people he was. You're supposed to write what you know, and he did that, indeed. And leaves the audience bored and confused, and jealous, for that matter. This is a curio for people who want to see Dorothy Stratten, who was murdered right after filming. But Patti Hanson, who would, in real life, marry Keith Richards, was also a model, like Stratten, but is a lot better and has a more ample part. In fact, Stratten's part seemed forced; added. She doesn't have a lot to do with the story, which is pretty convoluted to begin with. All in all, every character in this film is somebody that very few people 

In [6]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
from peft import LoraConfig, get_peft_model

In [7]:
# Load a pre-trained model for binary classification (num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The Warning we got is to be expected, because the classification head we added has not been pre-trained and the weights and biases have been newly initialized.

The bert-base-uncased model is a general-purpose, pre-trained BERT model. It has been trained on a large corpus of text using self-supervised objectives (like masked language modeling) but not for specific tasks like sentiment analysis, classification, etc.

In [8]:
# Apply LoRA for efficient fine-tuning
config = LoraConfig(r=8)
peft_model = get_peft_model(model, config)

`get_peft_model` is a function from the PEFT library that takes a pre-trained model and a LoRA configuration (`LoraConfig`) and returns a new model that has been adapted for parameter-efficient fine-tuning.
The new model (`peft_model`) has the same architecture as the original model (model) but with additional parameters introduced by LoRA that enable efficient fine-tuning.

In [9]:
# Load the IMDB dataset
dataset = load_dataset("imdb")

In [10]:
# Load a pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [11]:
# Define a function to tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

In [12]:
# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Set the format to include PyTorch tensors for input_ids, attention_mask, and label
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")  # Rename 'label' column to 'labels'
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [13]:
# Define training arguments
training_args = TrainingArguments(output_dir="./results",
                                  num_train_epochs=1,
                                  per_device_train_batch_size=32,
                                  report_to="none",
                                 )

In [14]:
# Initialize Trainer
trainer = Trainer(model=peft_model,
                  args=training_args,
                  train_dataset=tokenized_datasets['train'],
                  eval_dataset=tokenized_datasets['test'])

# Train the model
trainer.train()

/gpfs/opt/sw/jupyterhub/envs/conda/vsc5/jupyterhub-huggingface-v2/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss
500,0.681700


TrainOutput(global_step=782, training_loss=0.6659453409102262, metrics={'train_runtime': 196.9307, 'train_samples_per_second': 126.948, 'train_steps_per_second': 3.971, 'total_flos': 1650106406400000.0, 'train_loss': 0.6659453409102262, 'epoch': 1.0})

## Huggingface Hub

The Huggingface Hub is a platform for sharing models, datasets, and demos. It allows developers and researchers to collaborate, publish, and discover models and datasets, making the entire community's work more accessible.

### Key Features:
- A central repository for models, datasets, and metrics.
- Tools for versioning, collaboration, and deployment.
- Integrated with Huggingface libraries for easy use.

### Example: Uploading a Model to the Huggingface Hub
Here's how you can upload a model to the Huggingface Hub.

``` python
from huggingface_hub import HfApi

# Initialize the API
api = HfApi()

# Example of creating a new repository (Requires authentication)
repo_name = "my-awesome-model"
api.create_repo(repo_name)

# Save model locally and push to hub
model.save_pretrained(f"./{repo_name}")
api.upload_folder(repo_id=repo_name, folder_path=f"./{repo_name}")
```

### Explore Available Models

We will use the Huggingface API to search for available models and filter them based on certain criteria.


In [15]:
from huggingface_hub import HfApi

In [16]:
# Initialize the Huggingface API
api = HfApi()

In [17]:
# Search for models in the Huggingface Model Hub
models = list(api.list_models(limit=10))

In [18]:
# Display the fetched models
for model in models:
    print(model.modelId)

albert/albert-base-v1
albert/albert-base-v2
albert/albert-large-v1
albert/albert-large-v2
albert/albert-xlarge-v1
albert/albert-xlarge-v2
albert/albert-xxlarge-v1
albert/albert-xxlarge-v2
google-bert/bert-base-cased-finetuned-mrpc
google-bert/bert-base-cased


In [19]:
# Display the first 5 models for demonstration
models[:5]

[ModelInfo(id='albert/albert-base-v1', author=None, sha=None, created_at=datetime.datetime(2022, 3, 2, 23, 29, 4, tzinfo=datetime.timezone.utc), last_modified=None, private=False, gated=None, disabled=None, downloads=15530, likes=8, library_name='transformers', tags=['transformers', 'pytorch', 'tf', 'safetensors', 'albert', 'fill-mask', 'exbert', 'en', 'dataset:bookcorpus', 'dataset:wikipedia', 'arxiv:1909.11942', 'license:apache-2.0', 'autotrain_compatible', 'endpoints_compatible', 'region:us'], pipeline_tag='fill-mask', mask_token=None, card_data=None, widget_data=None, model_index=None, config=None, transformers_info=None, siblings=None, spaces=None, safetensors=None),
 ModelInfo(id='albert/albert-base-v2', author=None, sha=None, created_at=datetime.datetime(2022, 3, 2, 23, 29, 4, tzinfo=datetime.timezone.utc), last_modified=None, private=False, gated=None, disabled=None, downloads=1890109, likes=104, library_name='transformers', tags=['transformers', 'pytorch', 'tf', 'jax', 'rust',

Should you with to make that visually more pleasing, you can do so, by creating a pandas dataframe:

In [20]:
import pandas as pd

In [21]:
# Create the DataFrame by first creating a dictionary:
model_data = []

for model in models:
    model_info = {
        'Model ID': model.modelId,
        'Tags': ', '.join(model.tags) if model.tags else 'N/A',
        'Downloads': model.downloads,
        'Likes': model.likes,
        'Pipeline Tag': model.pipeline_tag if model.pipeline_tag else 'N/A',
        'Last Modified': model.lastModified.strftime('%Y-%m-%d') if model.lastModified else 'N/A'
    }
    model_data.append(model_info)


In [22]:
# Pass the dictionary to pandas DataFrame:
df_models = pd.DataFrame(model_data)

# Display the first 5 entries of the DataFrame:
df_models.head()

,Model ID,Tags,Downloads,Likes,Pipeline Tag,Last Modified
0,albert/albert-base-v1,"transformers, pytorch, tf, safetensors, albert...",15530,8,fill-mask,N/A
1,albert/albert-base-v2,"transformers, pytorch, tf, jax, rust, safetens...",1890109,104,fill-mask,N/A
2,albert/albert-large-v1,"transformers, pytorch, tf, albert, fill-mask, ...",1522,3,fill-mask,N/A
3,albert/albert-large-v2,"transformers, pytorch, tf, safetensors, albert...",11271,16,fill-mask,N/A
4,albert/albert-xlarge-v1,"transformers, pytorch, tf, safetensors, albert...",798,4,fill-mask,N/A


#### Analyze Model Information

Inspect the models to understand their details, such as architecture, number of parameters, and tasks.

In [23]:
# Display information about a specific model
model_name = "bert-base-uncased"  # Example model
model_info = api.model_info(model_name)

print(f"Model: {model_info.modelId}")
print(f"Description: {model_info.cardData.get('description', 'No description available')}")
print(f"Framework: {model_info.pipeline_tag}")
print(f"Tags: {model_info.tags}")

Model: google-bert/bert-base-uncased
Description: No description available
Framework: fill-mask
Tags: ['transformers', 'pytorch', 'tf', 'jax', 'rust', 'coreml', 'onnx', 'safetensors', 'bert', 'fill-mask', 'exbert', 'en', 'dataset:bookcorpus', 'dataset:wikipedia', 'arxiv:1810.04805', 'license:apache-2.0', 'autotrain_compatible', 'endpoints_compatible', 'region:us']


## Conclusion

In this notebook, we explored the Huggingface ecosystem, including the `transformers`, `datasets`, and Huggingface Hub.
We will get to know `PEFT` and `accelerate` in later notebooks.

In [24]:
# Shut down the kernel
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(restart=False)

{'status': 'ok', 'restart': False}